In [36]:
import numpy as np
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.callbacks import EarlyStopping, ReduceLROnPlateau

%matplotlib inline

In [4]:
train_dir = "spectrograms/"
test_dir = "test_spectrograms/"

In [18]:
trdata, testdata = ImageDataGenerator(rescale=1./255, validation_split=.2), ImageDataGenerator(rescale=1./255)
train_data = trdata.flow_from_directory(directory=train_dir,
                                        target_size= (256, 256),
                                        seed=42,
                                        subset="training")
val_data = trdata.flow_from_directory(directory=train_dir,
                                        target_size= (256, 256),
                                        seed=42,
                                        subset="validation")
test_data = testdata.flow_from_directory(directory=test_dir, target_size= (256, 256))

Found 23529 images belonging to 7 classes.
Found 5881 images belonging to 7 classes.
Found 7076 images belonging to 7 classes.


In [24]:
model = Sequential()

model.add(Conv2D(32, input_shape=(256, 256, 3), kernel_size=(5,5), padding="same", activation="relu"))
model.add(Conv2D(32, kernel_size=(5,5), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))

model.add(Conv2D(64, kernel_size=(5,5), padding="same", activation="relu"))
model.add(Conv2D(64, kernel_size=(5,5), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))

model.add(Conv2D(128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))

model.add(Conv2D(128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))

model.add(Flatten())

model.add(Dense(64, activation="relu"))
model.add(Dense(7, activation="softmax"))

In [25]:
from keras.optimizers import Adam
opt = Adam(lr=0.001)
model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [26]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_44 (Conv2D)           (None, 256, 256, 32)      2432      
_________________________________________________________________
conv2d_45 (Conv2D)           (None, 256, 256, 32)      25632     
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_46 (Conv2D)           (None, 128, 128, 64)      51264     
_________________________________________________________________
conv2d_47 (Conv2D)           (None, 128, 128, 64)      102464    
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_48 (Conv2D)           (None, 64, 64, 128)      

In [38]:
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=8, verbose=1, mode='auto')
reduce_lt = ReduceLROnPlateau(monitor="val_acc", patience=5, mode="auto", min_delta=0.0001, factor=0.5)

In [39]:
batch_size = 32
EPOCHS = 20

In [ ]:
hist = model.fit(train_data, validation_data=val_data,
                           batch_size=batch_size,
                           epochs=EPOCHS,
                           callbacks=[early, reduce_lt])

Epoch 1/20
